In [1]:
import tensorflow as tf
from tensorflow import keras

import os
import random
import tempfile
import pickle

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from keras.utils.np_utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import AUC, Recall, Precision
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
from keras import backend as K

In [2]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
path='drive/MyDrive/Capstone(Team10)/code'


file = tf.keras.utils

with open(path+'/data/X_data_full.pkl','rb') as f:
  X_data=pickle.load(f)

raw_df = pd.read_csv(path+'/data/rawdata_USA.csv', index_col=0, parse_dates=True)
raw_df.index.name='date'

metadata=pd.read_csv(path+'/data/metadata_final.csv')

X_data.head(5)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,103SPREAD,LCEAPR01,XTIMVA01,PRMNTO01,MABMM301,PRCNTO01,STMNIS01,SLWHTO02,ODMNTO02,XTEXVA01,...,IRSTFR01,IRLTLT01,IR3TCD01,BSPRTE02,LREMTTTT,LRHU24TT,LRUNTTTT,BSCURT02,3MTBILL,10YT
date,,,,,,,,,,,,,,,,,,,,,
1973-01-01,1.05,0.004975,0.047149,0.007775,0.009922,0.013699,-0.006349,0.040962,0.014942,0.084607,...,0.61,0.10,0.38,2.569360,-0.2,-1.1,-0.3,0.3890,0.34,0.10
1973-02-01,1.04,0.002478,0.044624,0.015954,0.004679,0.020691,-0.006390,0.016314,0.023197,0.022962,...,0.64,0.18,0.53,-6.665425,0.4,0.7,0.1,1.1154,0.19,0.18
1973-03-01,0.62,0.004938,-0.012775,0.001725,0.001473,0.014144,0.006390,0.019215,0.022260,0.046380,...,0.51,0.07,0.71,0.992926,0.3,-0.5,-0.1,-0.1107,0.49,0.07
1973-04-01,0.41,0.004914,-0.009876,-0.001709,0.005382,-0.027771,0.006349,0.021154,-0.008716,0.033822,...,0.03,-0.04,0.26,-3.104780,-0.1,0.6,0.1,-0.4168,0.17,-0.04
1973-05-01,0.49,0.004890,0.062043,0.006343,0.008624,-0.010260,0.000000,0.014101,0.006697,0.012248,...,0.72,0.18,0.24,-13.030239,0.0,-0.3,-0.1,0.2500,0.10,0.18


In [3]:
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []

    if type(Xs)==pd.core.frame.DataFrame:
      for i in range(len(X) - time_steps + 1):
          v = X.iloc[i:(i + time_steps)].values
          Xs.append(v)
          ys.append(y.iloc[i + time_steps-1].values)
          Xs=np.array(Xs)
          ys=np.array(ys)

    else:
      for i in range(len(X) - time_steps + 1):
          v = X[i:(i + time_steps)]
          Xs.append(v)
          ys.append(y[i + time_steps-1])

    return np.array(Xs), np.array(ys)

In [4]:
class BlockingTimeSeriesSplit():
    def __init__(self, n_splits):
        self.n_splits = n_splits

    def get_n_splits(self, groups):
        return self.n_splits

    def split(self, X, y=None, groups=None):
        n_samples = len(X)
        k_fold_size = n_samples // self.n_splits
        indices = np.arange(n_samples)

        margin = 0
        for i in range(self.n_splits):
            start = i * k_fold_size
            stop = start + k_fold_size
            mid = int(0.8 * (stop - start)) + start
            yield indices[start: mid], indices[mid + margin: stop]


In [5]:
cmap_data = plt.cm.Paired
cmap_cv = plt.cm.coolwarm
plt.style.use('fivethirtyeight')

## make index for train, validation dataset

def make_split(X_data, n_splits, test_year):
  train_idxs=[]
  val_idxs=[]

  tss=BlockingTimeSeriesSplit(n_splits=n_splits)

  for train_idx, val_idx in tss.split(X_data[:-(test_year*12)]):
    train_idxs.append(train_idx)
    val_idxs.append(val_idx)

  return train_idxs, val_idxs

def plot_cv_indices(cv, X, n_splits, lw=10):

    fig, ax = plt.subplots()
    """Create a sample plot for indices of a cross-validation object."""

    # Generate the training/testing visualizations for each CV split
    for ii, (tr, tt) in enumerate(cv.split(X=X)):
        # Fill in indices with the training/test groups
        indices = np.array([np.nan] * len(X))
        indices[tt] = 1
        indices[tr] = 0

        # Visualize the results
        ax.scatter(range(len(indices)), [ii + .5] * len(indices),
                   c=indices, marker='_', lw=lw, cmap=cmap_cv,
                   vmin=-.2, vmax=1.2)

    # Formatting
    yticklabels = list(range(n_splits))
    ax.set(yticks=np.arange(n_splits) + .5, yticklabels=yticklabels,
           xlabel='Sample index', ylabel="CV iteration",
           ylim=[n_splits+0.1, -.1], xlim=[0, len(X)])
    ax.set_title('{}'.format(type(cv).__name__), fontsize=15)

    ax.legend([Patch(color=cmap_cv(.8)), Patch(color=cmap_cv(.02))],
          ['Testing set', 'Training set'], loc=(1.02, .8))

In [6]:
def make_lstm_model(num_layer, X_train, nc, metrics, loss_type, dropout, activation, regularizer):
    model = Sequential()
    model.add(LSTM(num_layer, input_shape=(X_train.shape[1], X_train.shape[2]), dropout=dropout, activation=activation, kernel_regularizer=regularizer))  ##input_shape=(X_train.shape[1], X_train.shape[2])
    model.add(Dense(nc, activation='softmax'))
    model.compile(loss=loss_type, optimizer='adam', metrics=metrics)  ##['AUC','Accuracy','Recall','Precision','F1Score']
    return model

def make_gru_model(num_layer, X_train, nc, metrics, loss_type, dropout, activation, regularizer):
    model = Sequential()
    model.add(GRU(num_layer, input_shape=(X_train.shape[1], X_train.shape[2]), dropout=dropout, activation=activation, kernel_regularizer=regularizer))  ##input_shape=(X_train.shape[1], X_train.shape[2])
    model.add(Dense(nc, activation='softmax'))
    model.compile(loss=loss_type, optimizer='adam', metrics=metrics)  ##['AUC','Accuracy','Recall','Precision','F1Score']
    return model

In [7]:
######################run this cell only for training whole models####################################
np.random.seed(699)

os.environ['PYTHONHASHSEED'] = '699'
random.seed(699)

# Also set the tf randomness to some fixed values like this if you need:
tf.compat.v1.set_random_seed(699)
tf.keras.utils.set_random_seed(699)

session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

def grid_search(path, X_data, y_data, model_type, y_type, n_splits, test_year, nl=[30, 50, 70, 100],
                epochs=[100], batch_sizes=[12], time_steps=[12, 18, 24], es=True, dropouts=[0.2],  #
                activation='tanh', reg_methods=[None, 'L1','L1'], reg_factors=[0, 0.01, 0.03]):

#  results = []
#   history_dict={'model':[], 'val':[], 'history':[],
#                 'auc': [], #'acc': [], 'recall': [],'precision': [],
#                 'recall_0': [], \
#                 'recall_1': [], \
#                 'recall_2': [], \
#                 'final_auc':[], \
# #                'final_acc':[], \
# #                'final_recall':[], \
# #                'final_precision':[],
#                 'final_recall_0':[],
#                 'final_recall_1':[],
#                 'final_recall_2':[],}

#########tmp###############################(and comment out above history_dict part)
  with open(path+'/history_dict.pkl','rb') as f:
    history_dict=pickle.load(f)

  best_auc = 0
  best_params = None
  nc=len(y_data.unique())

  train_idxs, val_idxs = make_split(X_data, n_splits, test_year)

  for time_step in time_steps:
    loss_type = 'categorical_crossentropy'   ##if y_type == 'y_agg' else 'binary_crossentropy'
    for num_epochs in epochs:
      for batch_size in batch_sizes:
        for num_layer in nl:
          for dropout in dropouts:
            for (method, factor) in zip(reg_methods, reg_factors):
              if method=='L1':
                reg=regularizers.L1(factor)
              elif method=='L2':
                reg=regularizers.L2(factor)
              else:
                reg=None

              if method==None:
                model_name='clf_{}_{}_{}_b{}_ep{}_h{}_d{}_{}'.format(y_type, model_type, time_step, batch_size, num_epochs, num_layer, dropout, method)
              else:
                model_name='clf_{}_{}_{}_b{}_ep{}_h{}_d{}_{}_{}'.format(y_type, model_type, time_step, batch_size, num_epochs, num_layer, dropout, method, factor)


              for i, (train_idx, val_idx) in enumerate(zip(train_idxs, val_idxs)):

                X_train=X_data.iloc[list(train_idx)]
                y_train=y_data.iloc[list(train_idx)]
                X_val=X_data.iloc[list(val_idx)]
                y_val=y_data.iloc[list(val_idx)]

                ##standard scaling
                scaler = StandardScaler()
                X_train = scaler.fit_transform(X_train)
                X_val = scaler.transform(X_val)

                ##make timestep
                trainx, trainy = create_dataset(X_train, y_train, time_step) ##(X_train, y_train, time_step)
                valx, valy = create_dataset(X_val, y_val, time_step)   ##(X_val, y_val, time_step)
                trainy = to_categorical(trainy, num_classes=nc)
                valy = to_categorical(valy, num_classes=nc)

                ##assign class weight(more weight for less frequent class)
                if y_type!='y_agg':

                  metrics=[AUC(curve='ROC', multi_label=True, num_labels=2),  ##'Accuracy','Recall','Precision',
                          Recall(class_id=0, name='recall_0'),
                          Recall(class_id=1, name='recall_1')]

                else:

                  metrics=[AUC(curve='ROC', multi_label=True, num_labels=2),  ##'Accuracy','Recall','Precision',
                          Recall(class_id=0, name='recall_0'),
                          Recall(class_id=1, name='recall_1'),
                          Recall(class_id=2, name='recall_2')]



                # create the model weighted more for class 1
                if model_type=='lstm':
                  model=make_lstm_model(num_layer, trainx, nc, metrics, loss_type, dropout, activation=activation, regularizer=reg)
                else:
                  model=make_gru_model(num_layer, trainx, nc, metrics, loss_type, dropout, activation=activation, regularizer=reg)

                # assian early stopping and use best weight
                if es==True:
                  early_stop = EarlyStopping(monitor='recall_1', patience=5, verbose=1,
                                            restore_best_weights=True)   ##restore best_weight=True
                  history=model.fit(trainx, trainy, epochs=num_epochs, batch_size=batch_size, callbacks=[early_stop])
                else:
                  history=model.fit(trainx, trainy, epochs=num_epochs, batch_size=batch_size)

                # Final evaluation of the model
                scores = model.evaluate(valx, valy, verbose=0)

#                  print(valx, valy, scores)

                history_dict['model'].append(model_name)
                history_dict['val'].append(i)
                history_dict['history'].append(history)
                history_dict['auc'].append(scores[1])
#                history_dict['acc'].append(scores[2])
#                history_dict['recall'].append(scores[3])
#                history_dict['precision'].append(scores[4])
                history_dict['recall_0'].append(scores[2])
                history_dict['recall_1'].append(scores[3])

                if y_type=='y_agg':
                  history_dict['recall_2'].append(scores[4])
                else:
                  history_dict['recall_2'].append(-100)

              X_test=X_data.iloc[-test_year*12-time_step+1:]   ##revising this part: -time_step+1
              y_test=y_data.iloc[-test_year*12-time_step+1:]   ##revising this part: -time_step+1

              X_ft=X_data.iloc[:-test_year*12]
              y_ft=y_data.iloc[:-test_year*12]

              ##standard scaling
              scaler = StandardScaler()
              X_ft = scaler.fit_transform(X_ft)
              X_test = scaler.transform(X_test)

              ft_x, ft_y = create_dataset(X_ft, y_ft, time_step) ##(X_test, y_test, time_step)
              testx, testy = create_dataset(X_test, y_test, time_step) ##(X_test, y_test, time_step)
              ft_y = to_categorical(ft_y, num_classes=nc)
              testy = to_categorical(testy, num_classes=nc)

              ##assign class weight(more weight for less frequent class)
              if y_type!='y_agg':
                metrics=[AUC(curve='ROC', multi_label=True, num_labels=2),  ##'Accuracy','Recall','Precision',
                        Recall(class_id=0, name='recall_0'),
                        Recall(class_id=1, name='recall_1')]

              else:
                metrics=[AUC(curve='ROC', multi_label=True, num_labels=2),   ##'Accuracy','Recall','Precision',
                        Recall(class_id=0, name='recall_0'),
                        Recall(class_id=1, neame='recall_1'),
                        Recall(class_id=2, name='recall_2')]

              # create the model weighted more for class 1
              if model_type=='lstm':
                model=make_lstm_model(num_layer, ft_x, nc, metrics, loss_type, dropout, activation=activation, regularizer=reg)
              else:
                model=make_gru_model(num_layer, ft_x, nc, metrics, loss_type, dropout, activation=activation, regularizer=reg)


              if es==True:
                early_stop = EarlyStopping(monitor='recall_1', patience=5, verbose=1,
                                          restore_best_weights=True)   ##restore best_weight=True
                history=model.fit(ft_x, ft_y, epochs=num_epochs, batch_size=batch_size, callbacks=[early_stop])
              else:
                history=model.fit(ft_x, ft_y, epochs=num_epochs, batch_size=batch_size)


              final_scores=model.evaluate(testx, testy, verbose=0)  ##test...?


              # y_pred_prob=model.predict(testx)
              # y_pred=model.predict(testx).argmax(axis=1)
              # y_test=testy.argmax(axis=1)
              # cf_matrix = confusion_matrix(y_test, y_pred)
              # print(cf_matrix)
              # print(classification_report(y_test, y_pred))

              history_dict['final_auc'].extend([final_scores[1]]*n_splits)
#              history_dict['final_acc'].extend([final_scores[2]]*n_splits)
#              history_dict['final_recall'].extend([final_scores[3]]*n_splits)
#              history_dict['final_precision'].extend([final_scores[4]]*n_splits)
              history_dict['final_recall_0'].extend([final_scores[2]]*n_splits)
              history_dict['final_recall_1'].extend([final_scores[3]]*n_splits)
              if y_type=='y_agg':
                history_dict['final_recall_2'].extend([final_scores[4]]*n_splits)
              else:
                history_dict['final_recall_2'].extend([-100]*n_splits)

              with open(path+'/history_dict.pkl','wb') as f:
                pickle.dump(history_dict, f)

              model.save(path+'/{}.h5'.format(model_name))




In [8]:
######################run this cell only for training whole models####################################
ytype_list=['oecd']   #'oecd','oecd','oecd','oecd','nber','nber','nber','nber','agg','agg','agg','agg']
modeltype_list=['gru'] #'lstm','lstm','gru','gru','lstm','lstm','gru','gru','lstm','lstm','gru','gru']
es_list=[False]   #True, False, True, False, True, False, True, False, True, False, True, False]

cases=list(zip(ytype_list, modeltype_list, es_list))

for (ytype, model_type, es) in cases:

  np.random.seed(699)
  os.environ['PYTHONHASHSEED'] = '699'
  random.seed(699)
  # Also set the tf randomness to some fixed values like this if you need:
  tf.compat.v1.set_random_seed(699)
  tf.keras.utils.set_random_seed(699)

  session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
  sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
  tf.compat.v1.keras.backend.set_session(sess)

  threshold=50

  y_type='y_{}'.format(ytype)
  n_splits=3
  test_year=5

  y=raw_df[y_type]
  y_data=y[-(threshold*12+6):-6]

  if es==True:
    folder_path=path+'/model_test/change_test{}_{}_{}_es'.format(test_year, ytype, model_type)
  else:
    folder_path=path+'/model_test/change_test{}_{}_{}'.format(test_year, ytype, model_type)

  if not os.path.exists(folder_path):
    os.mkdir(folder_path)

  grid_search(folder_path, X_data, y_data, model_type, y_type, n_splits, test_year, nl=[20, 30, 40, 50, 70, 100], ##10,
              epochs=[100], batch_sizes=[12], time_steps=[24], es=es, dropouts=[0.2],   ##12, 18,
              activation='tanh', reg_methods=[None, 'L1','L1'], reg_factors=[0, 0.01, 0.03])

#path=folder_path
# nl=[30]
# epochs=[100]
# batch_sizes=[12]
# time_steps=[6, 9, 12, 18, 24]
# dropouts=[0.2]
# activation='tanh'
# reg_methods=['L1']
# reg_factors=[0.03]

Epoch 1/100
11/11 [==============================] - 3s 12ms/step - loss: 0.5703 - auc: 0.7672 - recall_0: 0.6825 - recall_1: 0.6379
Epoch 2/100
11/11 [==============================] - 0s 11ms/step - loss: 0.4588 - auc: 0.8799 - recall_0: 0.8571 - recall_1: 0.7586
Epoch 3/100
11/11 [==============================] - 0s 9ms/step - loss: 0.3591 - auc: 0.9418 - recall_0: 0.9048 - recall_1: 0.7759
Epoch 4/100
11/11 [==============================] - 0s 9ms/step - loss: 0.3126 - auc: 0.9572 - recall_0: 0.9683 - recall_1: 0.7759
Epoch 5/100
11/11 [==============================] - 0s 9ms/step - loss: 0.2631 - auc: 0.9762 - recall_0: 0.9683 - recall_1: 0.8448
Epoch 6/100
11/11 [==============================] - 0s 11ms/step - loss: 0.2486 - auc: 0.9751 - recall_0: 0.9524 - recall_1: 0.8621
Epoch 7/100
11/11 [==============================] - 0s 9ms/step - loss: 0.2076 - auc: 0.9866 - recall_0: 0.9683 - recall_1: 0.9138
Epoch 8/100
11/11 [==============================] - 0s 11ms/step - loss:

Epoch 1/100
11/11 [==============================] - 3s 17ms/step - loss: 3.8247 - auc_5: 0.6486 - recall_0: 0.6548 - recall_1: 0.5405
Epoch 2/100
11/11 [==============================] - 0s 18ms/step - loss: 3.5057 - auc_5: 0.8370 - recall_0: 0.7738 - recall_1: 0.7297
Epoch 3/100
11/11 [==============================] - 0s 13ms/step - loss: 3.2627 - auc_5: 0.8605 - recall_0: 0.8214 - recall_1: 0.5946
Epoch 4/100
11/11 [==============================] - 0s 14ms/step - loss: 3.0173 - auc_5: 0.9310 - recall_0: 0.9286 - recall_1: 0.6757
Epoch 5/100
11/11 [==============================] - 0s 15ms/step - loss: 2.7858 - auc_5: 0.9474 - recall_0: 0.9881 - recall_1: 0.6486
Epoch 6/100
11/11 [==============================] - 0s 14ms/step - loss: 2.6005 - auc_5: 0.9384 - recall_0: 0.9881 - recall_1: 0.5946
Epoch 7/100
11/11 [==============================] - 0s 17ms/step - loss: 2.3683 - auc_5: 0.9755 - recall_0: 1.0000 - recall_1: 0.6757
Epoch 8/100
11/11 [==============================] - 0s

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
44/44 [==============================] - 4s 17ms/step - loss: 14.8860 - auc_47: 0.6934 - recall_0: 0.7406 - recall_1: 0.5134
Epoch 2/100
44/44 [==============================] - 1s 17ms/step - loss: 8.5245 - auc_47: 0.8461 - recall_0: 0.8840 - recall_1: 0.6250
Epoch 3/100
44/44 [==============================] - 1s 18ms/step - loss: 4.0601 - auc_47: 0.8814 - recall_0: 0.9181 - recall_1: 0.6607
Epoch 4/100
44/44 [==============================] - 1s 18ms/step - loss: 1.5645 - auc_47: 0.9007 - recall_0: 0.8805 - recall_1: 0.7455
Epoch 5/100
44/44 [==============================] - 1s 18ms/step - loss: 0.8846 - auc_47: 0.8785 - recall_0: 0.8805 - recall_1: 0.7545
Epoch 6/100
44/44 [==============================] - 1s 15ms/step - loss: 0.7009 - auc_47: 0.8998 - recall_0: 0.9010 - recall_1: 0.7679
Epoch 7/100
44/44 [==============================] - 1s 17ms/step - loss: 0.6166 - auc_47: 0.9014 - recall_0: 0.8635 - recall_1: 0.7232
Epoch 8/100
44/44 [====

In [9]:
folders=[x for x in os.listdir(path+"/model_test") if ('change' in x)]  ##&('ls' not in x)

for i, folder in enumerate(folders):
  folder_path=path+'/model_test/{}'.format(folder)
  with open(folder_path+'/history_dict.pkl','rb') as f:
    history_dict=pickle.load(f)

  ytype=folder.split("_")[2]
  model_type=folder.split("_")[3]
  if folder.split("_")[-1]=='es':
    es=True
  else:
    es=False

  df_tmp=pd.DataFrame(history_dict)
  df_tmp['ytype']=ytype
  df_tmp['model_type']=model_type
  df_tmp['early_stopping']=es

  if i==0:
    df_result=df_tmp
  else:
    df_result=pd.concat([df_result, df_tmp])

## model selection: according to cross validation recall, auc scores
df_result['macro_recall']=(df_result['recall_0']+df_result['recall_1'])/2
df_selection=df_result.groupby(['model','early_stopping']).mean().sort_values(['macro_recall'], ascending=[False])

with open(path+'/model_test/df_result.pkl', 'wb') as f:
  pickle.dump(df_result, f)

with open(path+'/model_test/df_selection.pkl', 'wb') as f:
  pickle.dump(df_selection, f)

<ipython-input-9-e6d5ea6508c5>:27: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_selection=df_result.groupby(['model','early_stopping']).mean().sort_values(['macro_recall'], ascending=[False])


In [10]:
with open(path+'/model_test/df_selection.pkl', 'rb') as f:
  df_selection = pickle.load(f)

model_name=df_selection.iloc[0].name[0]
es=df_selection.iloc[0].name[1]
print(model_name, es)  ## best model=clf_y_oecd_gru_24_b12_ep100_h30_d0.2_None /  es=True

df_selection.head(10)

clf_y_oecd_lstm_24_b12_ep100_h10_d0.2_None True


,,val,auc,recall_0,recall_1,recall_2,final_auc,final_recall_0,final_recall_1,final_recall_2,macro_recall
model,early_stopping,,,,,,,,,,
clf_y_oecd_lstm_24_b12_ep100_h10_d0.2_None,True,1.0,0.634259,0.666667,0.602564,-100.0,0.870536,0.732143,1.00,-100.0,0.634615
clf_y_oecd_gru_18_b12_ep100_h10_d0.2_None,True,1.0,0.433333,0.518519,0.680702,-100.0,0.642857,0.803571,0.25,-100.0,0.599610
clf_y_oecd_gru_18_b12_ep100_h10_d0.2_L1_0.01,True,1.0,0.585185,0.629630,0.563158,-100.0,0.886161,0.589286,1.00,-100.0,0.596394
clf_y_oecd_lstm_12_b12_ep100_h30_d0.2_None,False,1.0,0.447917,0.666667,0.463333,-100.0,0.888393,0.732143,1.00,-100.0,0.565000
clf_y_oecd_gru_12_b12_ep100_h30_d0.2_None,True,1.0,0.464120,0.629630,0.476667,-100.0,0.863839,0.821429,0.75,-100.0,0.553148
clf_y_oecd_lstm_12_b12_ep100_h50_d0.2_L1_0.03,True,1.0,0.519676,0.666667,0.437500,-100.0,0.957589,0.696429,1.00,-100.0,0.552083
clf_y_oecd_lstm_18_b12_ep100_h70_d0.2_None,True,1.0,0.420370,0.666667,0.428070,-100.0,0.908482,0.714286,1.00,-100.0,0.547368
clf_y_oecd_lstm_12_b12_ep100_h20_d0.2_None,False,1.0,0.483796,0.666667,0.415833,-100.0,0.852679,0.785714,0.75,-100.0,0.541250
clf_y_oecd_gru_12_b12_ep100_h30_d0.2_None,False,1.0,0.431713,0.666667,0.410000,-100.0,0.872768,0.589286,1.00,-100.0,0.538333


In [11]:

# df_selection=df_selection.sort_values('macro_recall', ascending=False)

# with open(path+'/model_test/df_selection.pkl', 'wb') as f:
#   pickle.dump(df_selection, f)

In [12]:
#clf_y_oecd_lstm_24_b12_ep100_h10_d0.2_None